In [ ]:
!git clone https://github.com/alexa/Topical-Chat.git

Cloning into 'Topical-Chat'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 262 (delta 26), reused 15 (delta 3), pack-reused 197
Receiving objects: 100% (262/262), 96.25 MiB | 24.21 MiB/s, done.
Resolving deltas: 100% (134/134), done.
Updating files: 100% (119/119), done.


In [ ]:
!pip install transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer



In [ ]:
model_name = "gpt2"  # You can choose a different GPT-2 variant
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()



GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
def generate_response(user_input, max_length=100):
    input_ids = tokenizer.encode(user_input, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response


In [ ]:
print("Chatbot: Hello! How can I assist you today?")

while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break

    response = generate_response(user_input)
    print("Chatbot:", response)


In [ ]:
import json

train_dataset_path = "/content/Topical-Chat/conversations/train.json"
test_dataset_path="/content/Topical-Chat/conversations/test_freq.json"
test_validation_path="/content/Topical-Chat/conversations/valid_freq.json"



In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader, TensorDataset

# Define the chatbot model using Hugging Face's GPT-2
class ChatbotModel(nn.Module):
    def __init__(self):
        super(ChatbotModel, self).__init__()
        self.gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

    def forward(self, input_ids, attention_mask):
        return self.gpt2(input_ids, attention_mask=attention_mask)

# Preprocess and prepare your dataset (you'll need to load your dataset here)

# Tokenize your text data
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
input_ids = []
attention_masks = []

# Iterate through your dataset and tokenize the text
for conversation in your_dataset:
    conversation_tokens = tokenizer.encode(conversation, add_special_tokens=True, padding=True, truncation=True)
    input_ids.append(conversation_tokens)
    attention_masks.append([1] * len(conversation_tokens))

# Convert the tokenized data into PyTorch tensors
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

# Define hyperparameters
batch_size = 4
epochs = 10
learning_rate = 2e-5

# Create DataLoader for batch training
dataset = TensorDataset(input_ids, attention_masks)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the chatbot model
chatbot = ChatbotModel()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(chatbot.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    chatbot.train()

    for batch in dataloader:
        optimizer.zero_grad()

        input_ids, attention_mask = batch
        output = chatbot(input_ids, attention_mask=attention_mask, labels=input_ids)

        loss = criterion(output.logits.view(-1, output.logits.shape[-1]), input_ids.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}] - Average Loss: {avg_loss:.4f}")

# Save the trained chatbot model
chatbot.save_pretrained("chatbot_model")

# Inference (generate responses)
def generate_response(prompt, max_length=50):
    chatbot.eval()
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    response_ids = chatbot.generate(input_ids, max_length=max_length, num_return_sequences=1)
    response_text = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    return response_text

# Example usage
user_input = "Tell me about recent medical advancements."
response = generate_response(user_input)
print("Chatbot Response:", response)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW
from torch.utils.data import Dataset, DataLoader

# Define a custom dataset class to load your JSON data
class ConversationDataset(Dataset):
    def __init__(self, json_file_path, tokenizer):
        with open(json_file_path, 'r') as json_file:
            self.data = json.load(json_file)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        conversation = self.data[idx]
        # Combine messages into a single string
        input_text = " ".join([message["message"] for message in conversation])
        # Encode the text using the tokenizer
        input_ids = self.tokenizer.encode(input_text, add_special_tokens=True, max_length=512, truncation=True)
        return torch.tensor(input_ids)

# Define hyperparameters
batch_size = 4
epochs = 5
learning_rate = 2e-5

# Load your JSON data as a dataset
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
json_file_path = 'your_conversations.json'  # Replace with your JSON file path
dataset = ConversationDataset(json_file_path, tokenizer)

# Create a DataLoader for batch training
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the chatbot model
chatbot = GPT2LMHeadModel.from_pretrained("gpt2")

# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = AdamW(chatbot.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    chatbot.train()

    for batch in dataloader:
        optimizer.zero_grad()

        input_ids = batch.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        outputs = chatbot(input_ids, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}] - Average Loss: {avg_loss:.4f}")

# Save the trained chatbot model
chatbot.save_pretrained("chatbot_model")
